# Bayesian Model Selection

Authors: Yuchen Zhou and Audrey Olivier - 12/10/2018 <br>
Last modified by Audrey Olivier on 05/15/2020

## Quick review of Bayesian model selection

The problem of model selection consists in determining which model(s) best explain the available data $D$, given a set of candidate models $m_{1:M}$. Each model $m_{j}$ is parameterized by a set of parameters $\theta_{m_{j}} \in \Theta_{m_{j}}$, to be estimated based on data. In the Bayesian framework, model selection is perfomed by computing the posterior probability of each model $m_{j}$ using Bayes' theorem:

$$P(m_{j} \vert D) = \frac{p(D \vert m_{j})P(m_{j})}{\sum_{j=1}^{M} P(D \vert m_{j})P(m_{j})}$$

where $P(m_{j})$ is the prior assigned to model $m_{j}$ and $P(D \vert m_{j})$ is the model evidence, also called marginal likelihood.

$$ p(D \vert m_{j}) = \int_{\Theta_{m_{j}}} p(D \vert m_{j}, \theta_{m_{j}}) p(\theta_{m_{j}} \vert m_{j}) d\theta_{m_{j}} $$

where $p(\theta_{m_{j}} \vert m_{j})$ is the prior assigned to the parameter vector of model $m_{j}$.

## Numerical example

In the following we present an example for which the posterior pdf of the parameters, evidences and model probabilities can be computed analytically. We drop the $m_{j}$ subscript when referring to model parameters for simplicity. Three models are considered (the domain $x$ is fixed and consists in 50 equally spaced points):
\begin{align*}
m_{linear}:& \quad y = \theta_{0} x + \epsilon \\
m_{quadratic}:& \quad y = \theta_{0} x + \theta_{1} x^2 + \epsilon \\
m_{cubic}:& \quad y = \theta_{0} x + \theta_{1} x^2+ \theta_{2} x^3 + \epsilon \\
\end{align*}

All three models can be written in a compact form as $y=X \theta + \epsilon$, where $X$ contains the necessary powers of $x$. For all three models, the prior is chosen to be Gaussian, $p(\theta) = N(\cdot, \theta_{prior}, \Sigma_{prior})  $, and so is the noise $\epsilon \sim N(\cdot; 0, \sigma_{n}^{2} I)$. Then the posterior of the parameters can be computed analytically as:

\begin{align*}
& p(\theta \vert D={x,y}) =  N(\cdot; \theta_{post}(D), \Sigma_{post}(D)) \\
& \theta_{post}(D) = \left( \frac{1}{\sigma_{n}^{2}}X^{T}X + \Sigma_{prior}^{-1} \right)^{-1} \left(\frac{1}{\sigma_{n}^{2}}X^{T}y+\Sigma^{-1}\theta_{prior} \right) \\
& \Sigma_{post}(D) = \left( \frac{1}{\sigma_{n}^{2}}X^{T}X + \Sigma_{prior}^{-1} \right)^{-1}
\end{align*}

Then the evidence of each model can be computed as 

$$ p(D) = \frac{p(D \vert \theta)p(\theta)}{p(\theta \vert D)} $$
where $p(D \vert \theta) = N(\cdot; X\theta, \sigma_{n}^{2} I)$, $p(\theta) = N(\cdot, \theta_{prior}, \Sigma_{prior}) $ and $p(\theta \vert D) = N(\cdot, \theta_{post}(D), \Sigma_{post}(D))$. This formula can be computed at any point $\theta$.

### Generate data from the quadratic model

In [1]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Inference import *
from UQpy.RunModel import RunModel # required to run the quadratic model
from sklearn.neighbors import KernelDensity # for the plots
from statsmodels.nonparametric.kde import KDEUnivariate
from UQpy.Distributions import Normal

/Users/dimitris/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Generate data from a quadratic function
param_true = np.array([1.0, 2.0]).reshape(1, -1)
var_n = 1
error_covariance = var_n * np.eye(50)
print(param_true.shape)

z = RunModel(samples=param_true, model_script='pfn_models.py', model_object_name = 'model_quadratic', vec=False,
             var_names = ['theta_1', 'theta_2'])
data_clean = z.qoi_list[0].reshape((-1,))
data = data_clean + Normal(scale=np.sqrt(var_n)).rvs(nsamples=data_clean.size, random_state=456).reshape((-1,))
print(data)

(1, 2)
[ -0.6681285   -0.21082927   1.35993359   1.93062478   3.49961402
   4.73246234   4.52520653   5.95968726   6.61795398   8.26869254
   8.35470754  11.19380042  13.33213561  16.96838006  18.8585438
  23.74811764  26.5054303   28.4630389   31.5986224   35.10590689
  37.29487518  40.93165653  43.83484551  48.37478999  54.0178402
  57.30005362  61.24904528  65.93381664  71.12804783  77.11414629
  79.78168287  86.10212889  92.69324162  96.00700906 104.55678389
 108.35056844 115.97225104 121.67318536 128.39330449 133.44389061
 141.56142567 148.03345624 156.7101402  162.25358506 170.23398557
 177.57212657 186.03681389 194.80013151 201.7321561  210.44543202]


### Define the models, compute the true values of the evidence.

For all three models, a Gaussian prior is chosen for the parameters, with mean and covariance matrix of the appropriate dimensions. Each model is given prior probability $P(m_{j}) = 1/3$.

In [3]:
# Define the models
model_names = ['model_linear', 'model_quadratic', 'model_cubic']
model_n_params = [1, 2, 3]
model_prior_means = [[0.], [0., 0.], [0., 0., 0.]]
model_prior_stds = [[10.], [1., 1.], [1., 2., 0.25]]

In [4]:
evidences = []
model_posterior_means = []
model_posterior_stds = []
for n, model in enumerate(model_names):
    # compute matrix X
    X = np.linspace(0, 10, 50).reshape((-1,1))
    if n == 1: # quadratic model
        X = np.concatenate([X, X**2], axis=1)
    if n == 2: # cubic model
        X = np.concatenate([X, X**2, X**3], axis=1)

    # compute posterior pdf
    m_prior = np.array(model_prior_means[n]).reshape((-1,1))
    S_prior = np.diag(np.array(model_prior_stds[n])**2)
    S_posterior = np.linalg.inv(1/var_n*np.matmul(X.T,X)+np.linalg.inv(S_prior))
    m_posterior = np.matmul(S_posterior, 
                            1/var_n*np.matmul(X.T, data.reshape((-1,1)))+np.matmul(np.linalg.inv(S_prior),m_prior))
    m_prior = m_prior.reshape((-1,))
    m_posterior = m_posterior.reshape((-1,))
    model_posterior_means.append(list(m_posterior))
    model_posterior_stds.append(list(np.sqrt(np.diag(S_posterior))))
    print('posterior mean and covariance for '+model)
    print(m_posterior, S_posterior)
    
    # compute evidence, evaluate the formula at the posterior mean
    like_theta = multivariate_normal.pdf(data, mean=np.matmul(X,m_posterior).reshape((-1,)), cov=error_covariance)
    prior_theta = multivariate_normal.pdf(m_posterior, mean=m_prior, cov=S_prior)
    posterior_theta = multivariate_normal.pdf(m_posterior, mean=m_posterior, cov=S_posterior)
    evidence = like_theta*prior_theta/posterior_theta
    evidences.append(evidence)
    print('evidence for '+model+'= {}\n'.format(evidence))
    
# compute the posterior probability of each model
tmp = [1/3*evidence for evidence in evidences]
model_posterior_probas = [p/sum(tmp) for p in tmp]

print('posterior probabilities of all three models')
print(model_posterior_probas)

posterior mean and covariance for model_linear
[16.16834799] [[0.00059394]]


NameError: name 'multivariate_normal' is not defined

### Define the models for use in UQpy

In [ ]:
# Define models
from UQpy.Distributions import Normal, JointInd
candidate_models = []
for n, model_name in enumerate(model_names):
    run_model = RunModel(model_script='pfn_models.py', model_object_name=model_name, vec=False)
    prior = JointInd([Normal(loc=m, scale=std) for m, std in zip(model_prior_means[n], model_prior_stds[n])])
    model = InferenceModel(
        nparams=model_n_params[n], runmodel_object=run_model, prior=prior, error_covariance=error_covariance, 
        name=model_name)
    candidate_models.append(model)

### Run MCMC for one model

In [ ]:
# Quadratic model
from UQpy.SampleMethods import MH
bayesMCMC = BayesParameterEstimation(
    data=data, inference_model=candidate_models[1], sampling_class=MH, nsamples=3500, jump=10, nburn=100, 
    proposal=JointInd([Normal(scale=0.1), ] * 2), seed=[0., 0.], verbose=True, random_state=123)
# plot prior, true posterior and estimated posterior
fig, ax = plt.subplots(1,2,figsize=(16,5))
for n_p in range(2):
    domain_plot = np.linspace(-0.5,3,200)
    ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_prior_means[1][n_p], scale=model_prior_stds[1][n_p]),
                 label='prior', color='green', linestyle='--')
    ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_posterior_means[1][n_p], 
                                      scale=model_posterior_stds[1][n_p]),
                 label='true posterior', color='red', linestyle='-')
    ax[n_p].hist(bayesMCMC.sampler.samples[:, n_p], density=True, bins=30, label='estimated posterior MCMC')
    ax[n_p].legend()
    ax[n_p].set_title('MCMC for quadratic model')
plt.show(fig)

### Run Bayesian Model Selection for all three models

In [ ]:
# Defines constants for the MCMC learning part, same as above
proposals = [Normal(scale=0.1),
             JointInd([Normal(scale=0.1), Normal(scale=0.1)]),
             JointInd([Normal(scale=0.15), Normal(scale=0.1), Normal(scale=0.05)])]
nsamples = [2000, 6000, 14000]
nburn = [500, 2000, 4000]
jump = [5, 10, 25]

In [ ]:
selection = BayesModelSelection(
    data=data, candidate_models=candidate_models, prior_probabilities=[1./3., 1./3., 1./3.],
    nsamples=nsamples, sampling_class=[MH, ]*3, jump=jump, nburn=nburn, proposal=proposals,
    seed=model_prior_means, verbose=True, random_state=123)

In [ ]:
sorted_indices = np.argsort(selection.probabilities)[::-1]
print('Sorted models:')
print([selection.candidate_models[i].name for i in sorted_indices])
print('Evidence of sorted models:')
print([selection.evidences[i] for i in sorted_indices])
print('Posterior probabilities of sorted models:')
print([selection.probabilities[i] for i in sorted_indices])

As of version 2, the implementation of BayesModelSelection in UQpy uses the method of the harmonic mean to compute the models' evidence. This method is known to behave quite poorly, in particular it yeidls estimates with large variance. In the problem above, this implementation does not consistently detects that the quadratic model has the highest model probability. Future versions of UQpy will integrate more advanced methods for the estimation of the evidence.

In [ ]:
for i, (m, be) in enumerate(zip(selection.candidate_models, selection.bayes_estimators)):
    # plot prior, true posterior and estimated posterior
    print('Posterior parameters for model '+m.name)
    fig, ax = plt.subplots(1, 3, figsize=(16,5))
    for n_p in range(m.nparams):
        domain_plot = np.linspace(min(be.sampler.samples[:, n_p]), max(be.sampler.samples[:, n_p]), 200)
        ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_prior_means[i][n_p], 
                                           scale=model_prior_stds[i][n_p]),
                    label = 'prior', color='green', linestyle='--')
        ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_posterior_means[i][n_p], 
                                           scale=model_posterior_stds[i][n_p]),
                    label = 'true posterior', color='red', linestyle='-')
        ax[n_p].hist(be.sampler.samples[:, n_p], density=True, bins=30, label='estimated posterior MCMC')
        ax[n_p].legend()
    plt.show(fig)